<a href="https://colab.research.google.com/github/DajeanArcila/DajeanArcila/blob/main/Churn_clasifica%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Preveja os usuários com alta chance de deixar seu Streaming


In [371]:
#importamos as bibliotecas necesarias para manipulação, tratativa, clasificação e para medir as nosas metricas

#tratativa de dados
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.linear_model as lm
import requests
from io import StringIO


#medição de metricas
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error

#modelo de ML|RN
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier


In [372]:
df= pd.read_csv('/content/streaming_data.csv')
df.head()

,Age,Gender,Time_on_platform,Devices_connected,Subscription_type,Num_streaming_services,Num_active_profiles,Avg_rating,Churned,User_id
0,49.0,Male,4700.0,3.0,Basic,NaN,3,NaN,0.0,a271aa56-bcfc-4f0a-91f7-b773de6b86a4
1,69.0,Female,6107.0,2.0,Standard,NaN,1,2.14,0.0,fe14b048-770b-4619-8ec6-485b0969ae31
2,44.0,Male,6374.0,3.0,Premium,1.0,1,1.91,1.0,0283dffd-6684-4a05-9c6f-c02098a6849d
3,26.0,Male,NaN,5.0,Basic,NaN,2,NaN,NaN,c316f2f5-3403-4a0a-82c2-c98e4b3379d2
4,18.0,NaN,1948.0,3.0,Basic,NaN,3,3.88,0.0,14879bcb-1977-4ad8-a7f9-6aa9bf7b872f


In [373]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77884 entries, 0 to 77883
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Age                     66607 non-null  float64
 1   Gender                  63368 non-null  object 
 2   Time_on_platform        56464 non-null  float64
 3   Devices_connected       52696 non-null  float64
 4   Subscription_type       52663 non-null  object 
 5   Num_streaming_services  9572 non-null   float64
 6   Num_active_profiles     77884 non-null  int64  
 7   Avg_rating              60614 non-null  float64
 8   Churned                 61148 non-null  float64
 9   User_id                 77884 non-null  object 
dtypes: float64(6), int64(1), object(3)
memory usage: 5.9+ MB


In [374]:
df.isna().sum()

,0
Age,11277
Gender,14516
Time_on_platform,21420
Devices_connected,25188
Subscription_type,25221
Num_streaming_services,68312
Num_active_profiles,0
Avg_rating,17270
Churned,16736
User_id,0


In [375]:
#Posivelmente esta variavel nao afete muito a nossa analise, igual aqui apresento uma opcao para resolver este problema
'''
generos = ['Male', 'Female']

df['Gender'] = df['Gender'].apply(lambda x: np.random.choice(generos) if pd.isna(x) else x)
'''

"\ngeneros = ['Male', 'Female']\n\ndf['Gender'] = df['Gender'].apply(lambda x: np.random.choice(generos) if pd.isna(x) else x)\n"

In [376]:
df['Avg_rating'].fillna(0, inplace=True)

df['Num_streaming_services'].fillna(0, inplace=True)

df['Devices_connected'].fillna(0,inplace=True)

df['Time_on_platform'].fillna(0,inplace=True)

df['Churned'].fillna(0,inplace=True)



In [377]:
df.isna().sum()

,0
Age,11277
Gender,14516
Time_on_platform,0
Devices_connected,0
Subscription_type,25221
Num_streaming_services,0
Num_active_profiles,0
Avg_rating,0
Churned,0
User_id,0


In [378]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77884 entries, 0 to 77883
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Age                     66607 non-null  float64
 1   Gender                  63368 non-null  object 
 2   Time_on_platform        77884 non-null  float64
 3   Devices_connected       77884 non-null  float64
 4   Subscription_type       52663 non-null  object 
 5   Num_streaming_services  77884 non-null  float64
 6   Num_active_profiles     77884 non-null  int64  
 7   Avg_rating              77884 non-null  float64
 8   Churned                 77884 non-null  float64
 9   User_id                 77884 non-null  object 
dtypes: float64(6), int64(1), object(3)
memory usage: 5.9+ MB


In [379]:
df.isna().sum()

,0
Age,11277
Gender,14516
Time_on_platform,0
Devices_connected,0
Subscription_type,25221
Num_streaming_services,0
Num_active_profiles,0
Avg_rating,0
Churned,0
User_id,0


In [380]:
df=df['Churned'].replace({'Yes':1,'No':0})

In [384]:
# Convertir todas las columnas float a int
df = df.astype({'Time_on_platform': 'int', 'Devices_connected': 'int','Num_streaming_services':'int','Avg_rating':'int','Age':'int'})

KeyError: 'Only the Series name can be used for the key in Series dtype mappings.'

In [385]:
df.dropna()

,Churned
0,0.0
1,0.0
2,1.0
3,0.0
4,0.0
...,...
77879,0.0
77880,0.0
77881,1.0
77882,0.0
